In [3]:
import requests

In [4]:
r = requests.get("https://webs.iiitd.edu.in/cgibin/mhcbn/tapbinder_download.pl")

In [4]:
print(r.text)

<html><head><title>MHCBN: A comprehensive database of MHC binding and Non-binding peptides</title><base href="http://webs.iiitd.edu.in/raghava/mhcbn/"><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><style type="text/css"><!--a:active {  font-family: Verdana, Arial, Helvetica, sans-serif; font-size: 16px;  font-style: normal; font-weight: normal; font-variant: normal; text-transform: none; color: #000099; text-decoration: underline}a:hover {  font-family: Verdana, Arial, Helvetica, sans-serif; font-size: 12px;  font-style: normal; font-weight: normal; font-variant: normal; text-transform: capitalize; color: #FF0000; text-decoration: overline}a:link {  font-family: Verdana, Arial, Helvetica, sans-serif; font-size: 12px;  font-style: normal; font-weight: normal; font-variant: normal; text-transform: none; color: #0000CC; text-decoration: underline}a:visited {  font-family: Verdana, Arial, Helvetica, sans-serif; font-size: 12px; font-style: normal; line-height: nor

In [7]:
from lxml import etree

In [9]:
table = etree.HTML(r.text).find("body/table")
rows = iter(table)
headers = [col.text for col in next(rows)]
for row in rows:
    values = [col.text for col in row]
    print(dict(zip(headers, values)))

In [13]:
table

<Element table at 0x7f3b3868dac0>

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
       
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [self.parse_html_table(table) for table in soup.find_all('table')]  
    
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
        for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
            td_tags = row.find_all('td')     
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                        # Set the number of columns for our table
                    n_columns = len(td_tags)
                        
                # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
            # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                    
            # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [28]:
hp = HTMLTableParser()
url = "https://webs.iiitd.edu.in/cgibin/mhcbn/tapbinder_download.pl"
table = hp.parse_url(url)[1][2] # Grabbing the table from the tuple
table.head()

KeyError: 2

In [29]:
table

0                                                SEQUENCE
1       AYAKAAAAF                                     ...
2       GILGFVFTL                                     ...
3       GILGFVFTL                                     ...
4       QVPLRPMTYK                                    ...
                              ...                        
1049    PSSSWDFITV                                    ...
1050    FAPGNYPAL                                     ...
1051    FQPQNGQFI                                     ...
1052    KSPWFTTL                                      ...
1053    SGPSNTPPEI                                    ...
Name: 1, Length: 1054, dtype: object

In [6]:
tables = pd.read_html(r.text)

In [7]:
len(tables)

2

In [8]:
tables[0]

,0,1,2
0,NaN,FOUND 1053 ENTRIES ACCORDING TO YOUR SELECTED ...,NaN


In [9]:
tables[1]

,0,1
0,SNo,SEQUENCE
1,1,AYAKAAAAF
2,2,GILGFVFTL
3,3,GILGFVFTL
4,4,QVPLRPMTYK
...,...,...
1049,1049,PSSSWDFITV
1050,1050,FAPGNYPAL
1051,1051,FQPQNGQFI
1052,1052,KSPWFTTL


In [45]:
t = tables[1]
t.drop(index = 0,inplace = True)
t.rename(columns = {0:"id",1:"seq"},inplace = True)
t.head()

KeyError: '[0] not found in axis'

In [43]:
t.index

RangeIndex(start=1, stop=1054, step=1)

In [49]:
t.loc[2,:]

id             2
seq    GILGFVFTL
Name: 2, dtype: object

In [10]:
t = tables[1]
t.head()

,0,1
0,SNo,SEQUENCE
1,1,AYAKAAAAF
2,2,GILGFVFTL
3,3,GILGFVFTL
4,4,QVPLRPMTYK


In [11]:
t.iloc[0,0]

'SNo'

In [12]:
t1 = t.drop(index = 0)
t1.head()

,0,1
1,1,AYAKAAAAF
2,2,GILGFVFTL
3,3,GILGFVFTL
4,4,QVPLRPMTYK
5,5,SRYWAIRTR


In [13]:
t2 = t1.rename(columns = {0:"id",1:"seq"})
t2.head()

,id,seq
1,1,AYAKAAAAF
2,2,GILGFVFTL
3,3,GILGFVFTL
4,4,QVPLRPMTYK
5,5,SRYWAIRTR


In [14]:
t3 = t2.drop_duplicates(subset = "seq", ignore_index = True)
t3.shape

(470, 2)

In [15]:
def isCorrectSequence(seq):
    alphabet = list("ACDEFGHIKLMNPQRSTVWY")
    seq = str(seq).strip()
    for i in range(0,len(seq)):
        if seq[i] not in alphabet:
            return False
    return True
t3["isCorrectSeq"] = t3["seq"].apply(isCorrectSequence)
t3["isCorrectSeq"].value_counts()

/tmp/ipykernel_16526/3654482396.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t3["isCorrectSeq"] = t3["seq"].apply(isCorrectSequence)


True     467
False      3
Name: isCorrectSeq, dtype: int64

In [16]:
t4 = t3.loc[t3["isCorrectSeq"],:]
t4.shape

(467, 3)

In [17]:
url = "https://webs.iiitd.edu.in/cgibin/mhcbn/display.pl"
params = {"field":"sequence","value":"GLSRYVARL"}
r = requests.get(url, params = params)
print(r.text)

<center>
<br><h3>Found <font color ="red">8</font> Records in MHCBN</h3></center><STYLE type="text/css">A{text-decoration:none}---></STYLE><title>MHC Allele Query</title></head><html><head><title>MHC Allele Query</title></head><body bgcolor=#CCBE9D><form ENCTYPE="multipart/form-data" METHOD="POST" ACTION="/cgibin/mhcbn/mhc_sequence.pl"><br><html><head><title>MHC Allele Query</title></head><body bgcolor="#CCBE9D"><form ENCTYPE="multipart/form-data" METHOD="POST" ACTION="/cgibin/mhcbn/mhc_sequence.pl">
<table  border="5"  bordercolor="black" width="100%" cellpadding=6 cellspacing=10><br><tr><td width="30%">Entry No</td><td width="70%" colspan="2"><A href="/cgibin/mhcbn/display.pl?field=sno&value=32873">32873</td></tr><tr bgcolor="#CCBE9D"><td width="30%">MHC Allele</td><td width="70%"colspan="2" colspan="2"> <a href="/cgibin/mhcbn/display.pl?field=mhc_allele&value=TAP                                               ">TAP                                               &nbsp;</a></td></tr><pr

In [18]:
tt = pd.read_html(r.text)

In [19]:
len(tt)

8

In [22]:
k = tt[0].T
k.columns = k.iloc[0]
k1 = k.drop(index = 0)
k1

,Entry No,MHC Allele,Peptide Sequence,Host Organism,Anchor Position,Source Protein,Experimental Method,Binding Affinity,T cell Activity,MHC Class,Antigenic Structure,MHC Structure,Database Reference,Publication Reference,Source of Information,Comment
1,32873,TAP,GLSRYVARL,HUMAN,Not Determined,?,"CTL ASSAY, MHC AND TAP BINDING ASSAY",?,NaN,CLASS-1,NO VALUE OBSERVED,NaN,P17393&nbspQ02314&nbspP17395&nbspP03155&nbspP2...,10544195,Literature,Relative IC50(nM)=65
2,32873,TAP,GLSRYVARL,HUMAN,Not Determined,?,"CTL ASSAY, MHC AND TAP BINDING ASSAY",?,NaN,CLASS-1,NO VALUE OBSERVED,NaN,P17393&nbspQ02314&nbspP17395&nbspP03155&nbspP2...,10544195,Literature,Relative IC50(nM)=65


In [67]:
tt[1]

,0,1,2
0,Entry No,18620,18620
1,MHC Allele,HLA-A2,HLA-A2
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,No Value Observed,No Value Observed
5,Source Protein,?,?
6,Experimental Method,REFERENCE,REFERENCE
7,Binding Affinity,YES ?,YES ?
8,T cell Activity,?,?
9,MHC Class,CLASS-1,CLASS-1


In [68]:
tt[2]

,0,1,2
0,Entry No,32873,32873
1,MHC Allele,TAP,TAP
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,?,?
6,Experimental Method,"CTL ASSAY, MHC AND TAP BINDING ASSAY","CTL ASSAY, MHC AND TAP BINDING ASSAY"
7,Binding Affinity,?,?
8,T cell Activity,NaN,NaN
9,MHC Class,CLASS-1,CLASS-1


In [69]:
tt[3]

,0,1,2
0,Entry No,33010,33010
1,MHC Allele,TAP,TAP
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,?,?
6,Experimental Method,"CTL ASSAY, MHC AND TAP BINDING ASSAY","CTL ASSAY, MHC AND TAP BINDING ASSAY"
7,Binding Affinity,?,?
8,T cell Activity,?,?
9,MHC Class,CLASS-1,CLASS-1


In [70]:
tt[4]

,0,1,2
0,Entry No,21418,21418
1,MHC Allele,HLA-A*0202,HLA-A*0202
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,HBV POL (455-463),HBV POL (455-463)
6,Experimental Method,COMPETITION/CTL ASSAYS,COMPETITION/CTL ASSAYS
7,Binding Affinity,YES MODERATE,YES MODERATE
8,T cell Activity,YES ?,YES ?
9,MHC Class,CLASS-1,CLASS-1


In [71]:
tt[5]

,0,1,2
0,Entry No,21424,21424
1,MHC Allele,HLA-A*0203,HLA-A*0203
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,HBV POL (455-463),HBV POL (455-463)
6,Experimental Method,COMPETITION BINDING ASSAY,COMPETITION BINDING ASSAY
7,Binding Affinity,YES HIGH,YES HIGH
8,T cell Activity,?,?
9,MHC Class,CLASS-1,CLASS-1


In [72]:
tt[6]

,0,1,2
0,Entry No,21521,21521
1,MHC Allele,HLA-A*0205,HLA-A*0205
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,HBV POL (455-463),HBV POL (455-463)
6,Experimental Method,CTL ASSAYS,CTL ASSAYS
7,Binding Affinity,YES ?,YES ?
8,T cell Activity,YES ?,YES ?
9,MHC Class,CLASS-1,CLASS-1


In [73]:
tt[7]

,0,1,2
0,Entry No,21522,21522
1,MHC Allele,HLA-A*0206,HLA-A*0206
2,Peptide Sequence,GLSRYVARL,GLSRYVARL
3,Host Organism,HUMAN,HUMAN
4,Anchor Position,Not Determined,Not Determined
5,Source Protein,HBV POL (455-463),HBV POL (455-463)
6,Experimental Method,CTL ASSAYS,CTL ASSAYS
7,Binding Affinity,YES ?,YES ?
8,T cell Activity,YES ?,YES ?
9,MHC Class,CLASS-1,CLASS-1
